In [ ]:
# Note: we will be using jina embedding which has 8K context length hence useful with larger context
# Jina is based on the jinaBERT architecture, and supports symmetric bidirectional variant of ALiBi.

In [1]:
from transformers import AutoModel
from numpy.linalg import norm

In [2]:
cos_sim = lambda a,b : (a @ b.T) / (norm(a) * norm(b)) # @ is used for matrix multiplication

In [3]:
model = AutoModel.from_pretrained("jinaai/jina-embeddings-v2-base-en", trust_remote_code=True)
embeddings = model.encode(["How is the weather today", "What is the current weather like?"])

print(cos_sim(embeddings[0], embeddings[1]))

/Users/riaz/miniconda3/envs/cookbook/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

configuration_bert.py:   0%|          | 0.00/8.24k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_bert.py:   0%|          | 0.00/97.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

0.91080654


In [ ]:
### Note: the original demo uses HF inference endpoints but we are running this locally on a mac

In [23]:
# managing imports 
import asyncio
import json
from pathlib import Path
import time
from tqdm import tqdm

from typing import Optional
from datasets import load_dataset, Dataset, DatasetDict

import numpy as np
import pandas as pd

In [9]:
# We will create embeddings of a reddit dataset
DATASET_IN = "derek-thomas/dataset-creator-reddit-bestofredditorupdates"
DATASET_OUT = "data/processed-embeddings"

MAX_WORKERS = 5
ROW_COUNT = 100

In [10]:
dataset = load_dataset(DATASET_IN)

Generating train split:   0%|          | 0/11595 [00:00<?, ? examples/s]

In [11]:
# train data as pandas
documents = dataset["train"].to_pandas().to_dict("records")[:ROW_COUNT]

In [14]:
len(documents), documents[1]

(100,
 {'id': '10025jy',
  'content': '**I am not the original poster. Originally posted by**  [u/dude-wheres-my-car](https://www.reddit.com/user/dude-wheres-my-car/)  **in 2015 in** r/legaladvice\n\n&amp;#x200B;\n\n[**Car was mistakenly towed from my own parking spot and tow company won\'t release my car without paying huge storage fees**](https://www.reddit.com/r/legaladvice/comments/3x45v9/car_was_mistakenly_towed_from_my_own_parking_spot/) (16 Dec 2015)\n\nI own a condo that I am remodeling before renting it out again. The condo has 2 numbered parking spaces in an *oversized driveway*. My in-laws have been staying with us for the holidays, so we parked one of our cars (a black Land Cruiser) at the condo 2 weeks ago. Annoyingly, there was a dark red Toyota 4runner illegally parked there. We called the tow company contracted to our property and gave them the make/model, color, plate number, and the number of the spot. They said would tow the car. We parked our Land Cruiser in our oth

In [24]:
%%time
# Generating embeddings for the ROW_COUNT documents


for document in tqdm(documents):
    document["embedding"] = model.encode(document["content"])

 95%|█████████▌| 95/100 [05:14<00:33,  6.68s/it]